In [20]:
import gzip
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

### novozymes dataset sequences

In [2]:
df = pd.read_csv("/projects/robustmicrob/jlaw/inputs/novozymes/NOVO_train_repaired_df.csv")
df.head(2)

,WT_group,WT,position,MUT,sequence,mutant_seq,pH,df_sourse,data_source,source_df_id,tm,CIF,wt_tm,dTm
0,GP02,S,6,K,MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,MNQSVKSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,8.0,NESP train,10.1021/acscatal.9b05223,19480.0,62.5,AF-A0A2W0F5X5-F1,62.0,0.5
1,GP02,P,9,A,MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,MNQSVSSLAEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,8.0,NESP train,10.1021/acscatal.9b05223,19481.0,64.0,AF-A0A2W0F5X5-F1,62.0,2.0


In [49]:
# reformat the dataframe so that WT and mutant sequences are on their own lines
wt_df = df[['uniprot_id', 'WT_group', 'pH', 'wt_tm', 'data_source', 'sequence']].drop_duplicates()
len(wt_df)

30

In [4]:
df.pH.value_counts()

8.0     269
7.0     243
7.5     184
7.4      49
7.2      44
6.3      33
5.0      30
6.4      23
11.0     22
7.6      20
3.2      17
8.2      15
2.7      14
6.0      12
8.5      11
Name: pH, dtype: int64

### Map the sequence to the uniprot ID

In [5]:
# copied from here: https://stackoverflow.com/a/29805905/7483950
def read_fasta(fp, gzipped=False):
        name, seq = None, []
        for line in fp:
            if gzipped:
                line = line.decode()
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line[1:], []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))

In [6]:
seq_set = set(wt_df.sequence.values)

In [9]:
# find the uniprot IDs based on the sequences
fasta_file = "/projects/bpms/jlaw/inputs/uniprot/2023-01/uniprot_sprot_shortid.fasta.gz"
print(f"reading {fasta_file}")
seq_to_uniprot = {}
with gzip.open(fasta_file, 'r') as f:
    for name, seq in tqdm(read_fasta(f, gzipped=True)):
        if seq not in seq_set:
            continue
        seq_to_uniprot[seq] = name

print(f"\t{len(seq_to_uniprot)} found")

reading /projects/bpms/jlaw/inputs/uniprot/2023-01/uniprot_sprot_shortid.fasta.gz


568744it [00:04, 131340.38it/s]

	26 found


In [10]:
seq_set - set(seq_to_uniprot.keys())

{'MKFLQVLPALIPAALAQTSCDQWATFTGNGYTVSNNLWGASAGSGFGCVTAVSLSGGASWHADWQWSGGQNNVKSYQNSQIAIPQKRTVNSISSMPTTASWSYSGSNIRANVAYDLFTAANPNHVTYSGDYELMIWLGKYGDIGPIGSSQGTVNVGGQSWTLYYGYNGAMQVYSFVAQTNTTNYSGDVKNFFNYLRDNKGYNAAGQYVLSYQFGTEPFTGSGTLNVASWTASIN',
 'MMKMEGIALKKRLSWISVCLLVLVSAAGMLFSTAAKTETSSHKAHTEAQVINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPGKSIGGDIFSNREGKLPGKSGRTWREADINYTSGFRNSDRILYSSDWLIYKTTDAYQTFTKIR',
 'MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQGKGYIEAMAGLWSAALGFSNQRLIKAAEQQFNTLPFYHLFSHKSHRPSIELAEKLIEMAPVPMSKVFFTNSGSEANDTVVKMVWYLNNALGKPAKKKFISRVNGYHGITVASASLTGLPGNQRGFDLPLPGFLHVGCPHHYRFALAGESEEHFADRLAVELEQKILAEGPETIAAFIGEPLMGAGGVIVPPRTYWEKIQKVCRKYDILVIADEVICGFGRTGQMFGSQTFGIQPDIMVLSKQLSSSYQPIAAILINAPVFEGIADQSQALGALGHGFTGSGHPVATAVALENLKIIEEESLVEHAAQMGQLLRSGLQHFIDHPLVGEIRGCGLIAAVELVGDRVSKAPYQALGTLGRYMAGRAQEHGMITRAMGDAVAFCPPLIVNEQEVGMIVERFARALDDTTQWVG',
 'MVNLRNAVHSFLVHLIGLLVWQCDISVSPVAAIVTDIFNTSDGGRFKFPDGVQNWPALSIVIIIIMTIGGNILVIMAVSMEKKLHNATNYFLMSLAIADMLVGLLVMPLSLLAILYDYVWPLPRYLCPVWISLDVLF

In [35]:
seq_to_uniprot.update({
    'MKFLQVLPALIPAALAQTSCDQWATFTGNGYTVSNNLWGASAGSGFGCVTAVSLSGGASWHADWQWSGGQNNVKSYQNSQIAIPQKRTVNSISSMPTTASWSYSGSNIRANVAYDLFTAANPNHVTYSGDYELMIWLGKYGDIGPIGSSQGTVNVGGQSWTLYYGYNGAMQVYSFVAQTNTTNYSGDVKNFFNYLRDNKGYNAAGQYVLSYQFGTEPFTGSGTLNVASWTASIN': 'G0RRG8',
    'MMKMEGIALKKRLSWISVCLLVLVSAAGMLFSTAAKTETSSHKAHTEAQVINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPGKSIGGDIFSNREGKLPGKSGRTWREADINYTSGFRNSDRILYSSDWLIYKTTDAYQTFTKIR': 'P00648',
    # 'MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQGKGYIEAMAGLWSAALGFSNQRLIKAAEQQFNTLPFYHLFSHKSHRPSIELAEKLIEMAPVPMSKVFFTNSGSEANDTVVKMVWYLNNALGKPAKKKFISRVNGYHGITVASASLTGLPGNQRGFDLPLPGFLHVGCPHHYRFALAGESEEHFADRLAVELEQKILAEGPETIAAFIGEPLMGAGGVIVPPRTYWEKIQKVCRKYDILVIADEVICGFGRTGQMFGSQTFGIQPDIMVLSKQLSSSYQPIAAILINAPVFEGIADQSQALGALGHGFTGSGHPVATAVALENLKIIEEESLVEHAAQMGQLLRSGLQHFIDHPLVGEIRGCGLIAAVELVGDRVSKAPYQALGTLGRYMAGRAQEHGMITRAMGDAVAFCPPLIVNEQEVGMIVERFARALDDTTQWVG': '',
    'MVNLRNAVHSFLVHLIGLLVWQCDISVSPVAAIVTDIFNTSDGGRFKFPDGVQNWPALSIVIIIIMTIGGNILVIMAVSMEKKLHNATNYFLMSLAIADMLVGLLVMPLSLLAILYDYVWPLPRYLCPVWISLDVLFSTASIMHLCAISLDRYVAIRNPIEHSRFNSRTKAIMKIAIVWAISIGVSVPIPVIGLRDEEKVFVNNTTCVLNDPNFVLIGSFVAFFIPLTIMVITYCLTIYVLRRQALMLLHGHTEEPPGLSLDFLKCCKRNTAEEENSANPNQDQNARRRKKKERRPRGTMQAINNERKASKVLGIVFFVFLIMWCPFFITNILSVLCEKSCNQKLMEKLLNVFVWIGYVCSGINPLVYTLFNKIYRRAFSNYLRCNYKVEKKPPVRQIPRVAATALSGRELNVNIYRHTNEPVIEKASDNEPGIEMQVENLELPVNPSSVVSERISSV': 'Q5IS66',
})

In [36]:
df['uniprot_id'] = df.sequence.apply(lambda x: seq_to_uniprot.get(x, np.nan))
df.head(2)

,WT_group,WT,position,MUT,sequence,mutant_seq,pH,df_sourse,data_source,source_df_id,tm,CIF,wt_tm,dTm,uniprot_id
0,GP02,S,6,K,MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,MNQSVKSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,8.0,NESP train,10.1021/acscatal.9b05223,19480.0,62.5,AF-A0A2W0F5X5-F1,62.0,0.5,NaN
1,GP02,P,9,A,MNQSVSSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,MNQSVSSLAEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...,8.0,NESP train,10.1021/acscatal.9b05223,19481.0,64.0,AF-A0A2W0F5X5-F1,62.0,2.0,NaN


### reformat the dataframe so that WT and mutant sequences are on their own lines

In [47]:
mut_df = df[['uniprot_id', 'WT_group', 'WT', 'position', 'MUT', 'pH', 'tm', 'data_source', 'mutant_seq']]

In [52]:
mut_df.rename(columns={'mutant_seq': 'sequence'}, inplace=True)

/tmp/ipykernel_3539/1177040168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mut_df.rename(columns={'mutant_seq': 'sequence'}, inplace=True)


In [50]:
wt_df['tm'] = wt_df['wt_tm']
wt_df = wt_df.drop(columns=['wt_tm'])

In [53]:
seq_df = pd.concat([mut_df, wt_df])
seq_df.head(2)

,uniprot_id,WT_group,WT,position,MUT,pH,tm,data_source,sequence
0,NaN,GP02,S,6.0,K,8.0,62.5,10.1021/acscatal.9b05223,MNQSVKSLPEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...
1,NaN,GP02,P,9.0,A,8.0,64.0,10.1021/acscatal.9b05223,MNQSVSSLAEKDIQYQLHPYTNARLHQELGPLIIERGEGIYVYDDQ...


In [54]:
seq_df.tm.describe()

count    1016.000000
mean       55.018189
std        13.090118
min        22.000000
25%        45.375000
50%        54.925000
75%        62.000000
max        97.000000
Name: tm, dtype: float64

In [55]:
seq_df.to_csv("/projects/robustmicrob/jlaw/inputs/novozymes/processed/train_sequences.csv")